In [23]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness 

In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

## Vector Embedding And Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(text_documents,OpenAIEmbeddings())

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
import os
os.environ["USER_AGENT"] = "my-app/1.0"

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

In [ ]:
text_documents

In [ ]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()


from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
len(documents)

In [ ]:
docs = documents[:1]

In [ ]:
# ## Vector Embedding And Vector Store
# from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
# db = Chroma.from_documents(documents,OpenAIEmbeddings())

import ollama
import chromadb
client = chromadb.Client()
collection = client.create_collection(name="ollama_docs")

In [ ]:
from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="nomic-embed-text:latest"
)
texts = [d.page_content for d in documents] 
vector = embed.embed_documents(texts)
# print(vector)
len(vector)

In [ ]:
from langchain_community.vectorstores import Chroma

# Extract texts & metadata
texts = [d.page_content for d in documents]
metas = [d.metadata for d in documents]

# Initialize empty Chroma collection
vectordb = Chroma(
    collection_name="pdf_collection",
    embedding_function=embed,
    persist_directory="./chroma_store"
)

# Add precomputed embeddings
ids = [str(i) for i in range(len(texts))]
vectordb._collection.add(
    embeddings=vector,
    documents=texts,
    metadatas=metas,
    ids=ids
)

vectordb.persist()


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

# define embedding function
embed = OllamaEmbeddings(model="nomic-embed-text:latest")

# create Chroma vectorstore from documents
vectordb = Chroma.from_documents(
    documents=docs,                # your list of Document objects
    embedding=embed,
    persist_directory="./chroma_store"  # folder to save the DB
)

# persist to disk
vectordb.persist()


In [15]:
from sentence_transformers import SentenceTransformer

# Load embedding model on GPU
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device="cuda")

# Extract text from LangChain Document objects
texts = [doc.page_content for doc in documents]

# Encode text into embeddings
embeddings = model.encode(texts, show_progress_bar=True, convert_to_numpy=True)

print(f"Generated {len(embeddings)} embeddings of size {embeddings.shape[1]}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\SOHEL\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Generated 10 embeddings of size 1024


In [19]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

# 1. Load PDF
loader = TextLoader("speech.txt")
documents = loader.load()

# 2. Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# 3. Define embedding function using SentenceTransformer
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device="cuda")

def embedding_function(texts):
    return model.encode(texts, convert_to_numpy=True, show_progress_bar=True)

# 4. Store in Chroma
db = Chroma.from_documents(
    docs,
    embedding_function,          # only pass once here
    persist_directory="./chroma_db"
)

db.persist()
print("✅ PDF embedded and stored in Chroma successfully!")


AttributeError: 'function' object has no attribute 'embed_documents'

In [ ]:
query = "Who are the authors of attention is all you need?"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

## Ollama Embedding models

In [ ]:
import ollama
import chromadb

In [ ]:
client = chromadb.Client()
collection = client.create_collection(name="ollama_docs")

collection

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()
docs

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
len(documents)

docs = documents[:10]

In [ ]:
# store each document in a vector embedding database
for i, d in enumerate(docs):
  response = ollama.embed(model="nomic-embed-text", input=d.page_content)
  embeddings = response["embeddings"]
  collection.add(
    ids=[str(i)],
    embeddings=embeddings,
    documents=[d.page_content]
  )